## Concurrency vs. Parallelism

----

- Concurrency(동시성): 디스크, 네트워크 또는 단일 CPU 코어와 같은 동일한 공유 리소스에 번갈아가는 접근을 허용하는 작업
    - 목표: 외부 자원(디스크, 네트워크, CPU 등)에 대하여 대기해야할 때, 작업을 전환하는 방법. ex. 여러 네트워크 요청
- Parallelism(병렬성): 여러 CPU 코어와 같이 독립적으로 분할된 리소스에서 여러 작업을 나란히 실행할 수 있도록 하는 것
    - 목표: 하드웨어 자원의 사용을 극대화 하는 것. ex. 1개의 CPU에게 8가지 일 → 8개의 CPU에게 동시에 1가지 일
- 파이썬
    - 파이썬에서는 Concurrency를 위해 `threading`과 `async`(coroutines)을 지원하며, 둘은 많은 요소들을 공유한다.
    - Parallelism을 위해 `multiprocessing`을 통해 여러 python interpreter를 실행할 수 있게 한다.
    - threading과 coroutines은 항상은 아니지만 대체될 수 있으며, multiprocessing은 CPU 활용률을 최대화 하고 싶을 때 사용된다.


### 1. Python Threading

---

- 예시
    - 여러 URL의 데이터를 한번에 읽기
    - 직접 스레들 생성하는 대신 `ThreadPoolExecutor`를 사용
        - Pool: 사용할 때 획득한 메모리나 나중에 해제되는 메모리가 아닌, **사용할 준비가 된 메모리에 유지되는 리소스의 모음** 
            - 풀 클라이언트는 풀에서 자원를 요청해 작업을 수행. 작업을 마치면, 풀로 반환.
        - 특징
            - 풀링은 높은 리소스 획득 비용이 크거나, 리소스 요청이 많거나, 리소스가 적은 상황에서 응답 시간을 늘릴 수 있다.
            - 풀링은 리소스를 이미 확보했기 때문에, latency를 고려할 때 유용.
            - 따라서, 시스템 리소스가 필요한 시스템콜이나 원격 리소스가 필요한 DB연결, 소켓 연결, 스레드, 메모리할당 등에 적용할 수 있다.
            - 이외에도 풀링은 연산량이 큰 데이터를 연산할 때 (그래픽, 데이터캐싱, 메모아이제이션)에도 유용하다.
            - 예시, [Connection Pools](https://en.wikipedia.org/wiki/Connection_pool), [Thread Pools](https://en.wikipedia.org/wiki/Thread_pool), [Memory Pools](https://en.wikipedia.org/wiki/Memory_pool)
            
        - [Ref](https://en.wikipedia.org/wiki/Pool_(computer_science)
        
        

    

- Python의 Thread
    - CPython에서 파이썬 Thread는 OS 스레드이다. 파이썬 런타임에 의해 필요에 따라 서로에게 양보하여 협력적으로 실행되도록 관리된다.
    - 장점
        - 파이썬의 스레드는 다른 리소스에서 대기하는 작업을 실행하는 편리하고 잘 이해된 방법을 제공함.
        - 아래와 같은 네트워크 호출 이외에도 하드웨어 장치, main thread의 signal을 threading 할 수 있음.
    - 단점
        - thread는 협력적(cooperative). 파이썬 런타임은 thread에 의해 접근하는 대상을 올바르게 관리할 수 있도록 attention을 2개로 나눔.(계속 확인한다?)
        - CPU를 많이 사용하는 작업을 Threading하는 경우, 파이썬 런타임은 쓰레드 전환 시 일시 중지되므로 Thread 외부에서 작업을 하는 이점을 얻지 못함.
        - 서로 다른 쓰레드에서 데이터를 접근한다면, 수동으로 동기화를 통해 우리가 예상하는 결과를 얻게 해야 함.
        



In [1]:
from concurrent.futures import ThreadPoolExecutor
import urllib.request as ur

datas = []

def get_from(url):
    connection = ur.urlopen(url)
    data = connection.read()
    datas.append(data)

urls = [
    "https://python.org",
    "https://docs.python.org/"
    "https://wikipedia.org",
    "https://imdb.com",    
]

with ThreadPoolExecutor() as ex:
    for url in urls:
        ex.submit(get_from, url)

# let's just look at the beginning of each data stream
# as this could be a lot of data
print([_[:200] for _ in datas])

[b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <h', b'<!DOCTYPE html><html lang="en-US" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml"><head><meta name="viewport" content="width=device-width"/><meta charSet="']


### 2. Python Coroutines and `async`

---

- 예시
    - 아래는 위 예시를 비동기/코루틴 구조로 변경한 것이다.
    - 아래의 `get_from` 함수는 다른 코루틴과 함께 실행될 수 있는 함수이다.
    - `asyncio.gather`는 여러 코루틴을 실행하고 모두 실행될 때까지 기다린 후 집계된 데이터를 반환한다.
    - 일반적으로 많이 사용하는 `urllib.request`는 비동기적인 요청을 지원하지 않는다.
    - `aiohttp` 라이브러리는 비동기적으로 네트워크 연결을 가능하게 한다.


-  Python Coroutines(async)
    - 코루틴, 비동기는 시스템 스레드가 아닌 특별한 프로그래밍 구조를 통해 동시성있게(concurrently) 함수를 실행한다.
    - 코루틴은 파이썬 런타임에의해 관리되지만, 스레드보다 오버헤드가 훨씬 적다.
    - 장점
        - 프로그램의 함수를 나란히 실행한다는 구문이 명확함.(async를 사용하고 쓰레드는 모든 함수가 스레드에서 실행되기 때문에)
        - 구조적 제한이 거의 없다. 코루틴간의 전환은 오버헤드가 적고 스레드보다 적은 메모리가 필요. 파이썬 런타임에서 직접 관리할 수 있어, 스레드가 필요하지 않지만, 필요한 경우 별도 스레드에서 실행 가능.
    - 단점
        - 고유한 구문 사용(`async def` , `await`)으로 인한, 동기 코드와 혼합이 어려움.
        - CPU 집약적인 작업을 효율적으로 나란히 실행할 수 없음. 스레드와 마찬가지로, 외부조건에 따라 대기해야하는 작업을 위해 설계됨.


In [2]:
# %load ./python_coroutines.py
# import aiohttp
# import asyncio

# urls = [
#     "https://imdb.com",    
#     "https://python.org",
#     "https://docs.python.org",
#     "https://wikipedia.org",
# ]

# async def get_from(session, url):
#     async with session.get(url) as r:
#         return await r.text()


# async def main():
#     async with aiohttp.ClientSession() as session:
#         datas = await asyncio.gather(*[get_from(session, u) for u in urls])
#         print ([_[:200] for _ in datas])

# if __name__ == "__main__":
#     loop = asyncio.get_event_loop()
#     loop.run_until_complete(main())

### 3. Python Multiprocessing
---


- 예시
    - `Pool`: 재사용이 가능한 프로세스 그룹
    - `map`: 각 프로세스에서 실행할 함수와 iterable한 인스턴스(URL 목록)을 전송
    - 위 예시들과의 차이점은 `get_from`함수에서 CPU바인딩 작업을 수행한다는 것
    - 또한, 정규표현식으로 찾는 작업 추가했듯이 비교적 비용이 많이드는 작업 수행 가능


- Python Multiprocessing
    - 파이썬 런타임의 독립적인 여러 복사본에서 CPU 집약적인 작업을 나란히 실행할 수 있음
    - 각 파이썬 런타임은 해당 작업을 실행하는데 필요한 코드와 데이터를 받음
    - 장점
        - 스레딩과 코루틴은 작업을 강제로 직렬화하기 때문에, 파이썬 object에 대한 접근을 관리해야 함. 멀티프로세싱은 각 작업에 별도의 파이썬 런타임과 CPU코어를 제공해 이러한 제한을 회피함.
    - 단점
        - 프로세스 생성과 관련된 추가 오버헤드. 이는 프로세스를 함수가 응용프로그램(함수?)의 수명동안 한번 회전 시키고 다시 사용하면 오버헤드의 영향을 최소화할 수 있음.
        - 각 하위 프로세스가 작업하는 데이터의 복사본을 주 프로세스에 전송해야한다는 것. 데이터 전송을 위해 파이썬 object를 이진 형태로 나타낼 수 있도록 하는 pickle 프로토콜을 사용하며, numbers, strings, lists, dictionaries, tuples, bytes 등 이외의 데이터 형태는 전송할 수 없음.
    

In [3]:
import urllib.request as ur
from multiprocessing import Pool
import re

urls = [
    "https://python.org",
    "https://docs.python.org",
    "https://wikipedia.org",
    "https://imdb.com",    
]

meta_match = re.compile("<meta .*?>")

def get_from(url):
    connection = ur.urlopen(url)
    data = str(connection.read())
    return meta_match.findall(data)


with Pool() as p:
    datas = p.map(get_from, urls)
print (datas)


[['<meta charset="utf-8">', '<meta http-equiv="X-UA-Compatible" content="IE=edge">', '<meta name="application-name" content="Python.org">', '<meta name="msapplication-tooltip" content="The official home of the Python Programming Language">', '<meta name="apple-mobile-web-app-title" content="Python.org">', '<meta name="apple-mobile-web-app-capable" content="yes">', '<meta name="apple-mobile-web-app-status-bar-style" content="black">', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', '<meta name="HandheldFriendly" content="True">', '<meta name="format-detection" content="telephone=no">', '<meta http-equiv="cleartype" content="on">', '<meta http-equiv="imagetoolbar" content="false">', '<meta name="msapplication-TileImage" content="/static/metro-icon-144x144-precomposed.png">', '<meta name="msapplication-TileColor" content="#3673a5">', '<meta name="msapplication-navbutton-color" content="#3673a5">', '<meta name="description" content="The official home of the Python

## References
---
- https://www.infoworld.com/article/3632284/python-concurrency-and-parallelism-explained.html
